# User Satsfaction Score Prediction

In [5]:
import pandas as pd
import os
import sys
# Add the 'scripts' directory to the Python path for module imports
sys.path.append(os.path.abspath(os.path.join('..', 'scripts')))

In [6]:
# Load the xdr_cleaned dataset
data = pd.read_csv('../data/xdr_cleaned.csv')

In [7]:
# Import modules
from user_engagement_analysis import UserEngagementAnalysis
from experience_analytics import ExperienceAnalytics
from satisfaction_analytics import UserSatisfactionAnalytics

Connection to database successful!


In [9]:
# Create instances of engagement and user experience classes
engagement_analysis = UserEngagementAnalysis(data)
experience_analysis = ExperienceAnalytics(data)
satisfaction_analysis = UserSatisfactionAnalytics()

In [10]:
# Normalize and cluster
# Aggregate engagement metrics
engagement_analysis.aggregate_metrics()
engagement_analysis.metrics.head()
engagement_analysis.normalize_and_cluster(n_clusters=3)

In [ ]:
# Computer Engagement Score
engagement_features = [
    'total_session_duration',
    'total_download_traffic',
    'total_upload_traffic', 
    'sessions_frequency']

# Identify least engaged cluster
least_engaged_cluster = engagement_analysis.cluster_centers[engagement_features].sum(axis=1).idxmin()

# Compute engagement scores
engagement_scores = satisfaction_analysis.compute_score(
    engagement_analysis.metrics, 
    engagement_analysis.cluster_centers, 
    engagement_features, 'Engagement_Score',
    least_engaged_cluster)